In [37]:
# coding: utf-8
import requests
import csv
from bs4 import BeautifulSoup

def get_the_page(url): #get one page's restaurants info
    
    r = requests.get(url,headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36'})
    html_str = r.text
    mypage = BeautifulSoup(html_str,"html.parser")
    
    myitems = mypage.find_all('li',attrs={'class':'sr1-listing-content-cell pois-restaurant-list-cell '})
    
    titles=[]
    likes=[]
    locations=[]
    prices=[]
    countries=[]
    styles=[]
    reviews=[]
    bookmarks=[]
    discounts=[]
    
    for myitem in myitems:
        
        titles.append(myitem.find('h2').text.strip())
             
        like=myitem.find('span',attrs={'class':'score score-big highlight'})
        if like is not None:       #some item doesn't have like or other features
            likes.append(like.text)
        else:
            like = '0'
            likes.append(like)
        
        location = myitem.find('div',attrs={'class':'icon-info address'})
        if location is not None:      
            locations.append(location.find('span').text.strip())
        else:
            location = 'NONE'
            locations.append(location)
        
        price = myitem.find('div',attrs={'class': 'icon-info icon-info-food-price'})
        if price is not None:       
            prices.append(price.find('span').text)
        else:
            price = 'NONE'
            prices.append(price)
        
        review = myitem.find('div',attrs={'class':'counters-container'})
        if review is not None:       
            reviews.append(review.find('span').text)
        else:
            review = '0'
            reviews.append(review)
        
        bookmark = myitem.find('div',attrs={'class':'text bookmarkedUserCount js-bookmark-count'})
        if bookmark is not None:       
            bookmarks.append(bookmark['data-count'])
        else:
            bookmark = '0'
            bookmarks.append(bookmark)

        discount = myitem.find('div',attrs={'class':'info-row'})
        if discount is not None:       
            discounts.append(discount.find('span').text)
        else:
            discount = 'NONE'
            discounts.append(discount)
            
    countries_materials=mypage.find_all("ul",attrs={'class':'pois-categoryui-list'})
    country_material=[]
    for material in countries_materials:
        country_material.append(material.find_all('li'))
    for li in country_material:
        countries.append(li[0].text.strip())
        if len(li)>1:
            styles.append(li[1].text.strip())
        else:
            styles.append(li[0].text.strip())
    
    print(url)
    
    with open('openrice_sample.csv', 'a') as f:
        writer = csv.writer(f)
        for i in range(0,len(titles)):
            writer.writerow([titles[i],locations[i],prices[i],countries[i],styles[i],likes[i],reviews[i],bookmarks[i],discounts[i]])
    return

In [38]:
def scrape_pages(url_page1):    #get 17 pages, which is limit of open rice, of the results return from your search
    get_the_page(url_page1)
    for q in range(1,17):
        page_number=str(q+1)
        url_next='{0}&page={1}'.format(url_page1,page_number)
        get_the_page(url_next)
    return

In [39]:
url_list = ['https://www.openrice.com/zh/hongkong/restaurants?Seat=2&BookingDate=2018-09-30&BookingPeriod=Breakfast',
            'https://www.openrice.com/zh/hongkong/restaurants?Seat=2&BookingDate=2018-09-30&BookingPeriod=Lunch',
            'https://www.openrice.com/zh/hongkong/restaurants?Seat=2&BookingDate=2018-09-30&BookingPeriod=Tea',
            'https://www.openrice.com/zh/hongkong/restaurants?Seat=2&BookingDate=2018-09-30&BookingPeriod=Dinner']

In [40]:
fail = []
for url in url_list:
    try: 
        scrape_pages(url)
    except Exception as e:
        print(e)
        print(url)
        fail.append((url, e))

https://www.openrice.com/zh/hongkong/restaurants?Seat=2&BookingDate=2018-09-30&BookingPeriod=Breakfast
https://www.openrice.com/zh/hongkong/restaurants?Seat=2&BookingDate=2018-09-30&BookingPeriod=Breakfast&page=2
https://www.openrice.com/zh/hongkong/restaurants?Seat=2&BookingDate=2018-09-30&BookingPeriod=Breakfast&page=3
https://www.openrice.com/zh/hongkong/restaurants?Seat=2&BookingDate=2018-09-30&BookingPeriod=Breakfast&page=4
https://www.openrice.com/zh/hongkong/restaurants?Seat=2&BookingDate=2018-09-30&BookingPeriod=Breakfast&page=5
https://www.openrice.com/zh/hongkong/restaurants?Seat=2&BookingDate=2018-09-30&BookingPeriod=Breakfast&page=6
https://www.openrice.com/zh/hongkong/restaurants?Seat=2&BookingDate=2018-09-30&BookingPeriod=Breakfast&page=7
https://www.openrice.com/zh/hongkong/restaurants?Seat=2&BookingDate=2018-09-30&BookingPeriod=Breakfast&page=8
https://www.openrice.com/zh/hongkong/restaurants?Seat=2&BookingDate=2018-09-30&BookingPeriod=Breakfast&page=9
https://www.openr

In [46]:
import pandas
df = pandas.read_csv('openrice_sample.csv',header=None, names=['name', 'location', 'price', 'country', 'style', 'likes','review','bookmark','discount_info']) 

In [47]:
len(df)

1198

In [48]:
df = df.drop_duplicates() #remove duplicated rows

In [49]:
len(df) #i guess its because some restaurants are opening all day

370

In [50]:
df

,name,location,price,country,style,likes,review,bookmark,discount_info
0,Yadllie Plate,旺角西洋菜街1號兆萬中心11樓,$101-200,西式,海鮮,487,(630 食評),27495,早鳥及消夜時段訂座 可享全單八折優惠
1,Sky726,旺角彌敦道724-726號25樓,$201-400,韓國菜,韓式炸雞,367,(494 食評),38413,自選餐牌主菜買一送一優惠 或 甜品8折優惠； 飲品一律8折
2,Plato Cafe & Bistro,油麻地彌敦道483-485號2樓,$101-200,法國菜,甜品/糖水,408,(503 食評),22827,訂座可贈送小食
3,鳥居居酒屋 tori izakaya,銅鑼灣軒尼詩道525號澳門逸園中心17樓,$201-400,西式,甜品/糖水,412,(471 食評),14133,送 25里數 / 30積分
4,Ocio,旺角煙廠街33號朗廷軒地下B舖,$101-200,日本菜,爐端燒,432,(613 食評),18250,送 25里數 / 30積分
5,孖啡 Double Cafe,旺角彌敦道577號高氏大樓4樓,$51-100,多國菜,甜品/糖水,295,(384 食評),20576,79折
6,EAT ME Bar & Restaurant,旺角彌敦道560號譽發廣場5樓全層,$101-200,西式,甜品/糖水,481,(657 食評),20493,全單8折
7,Pom's Kitchen & Deli,佐敦德興街7號A-B號舖,$101-200,多國菜,咖啡店,268,(361 食評),18496,全單88折
8,Le Viet,尖沙咀彌敦道100號The ONE 4樓L402舖,$101-200,西式,西餐廳,309,(391 食評),9041,送 25里數 / 30積分
9,第三站 Station 3 Cafe,旺角洗衣街39號金雞廣場25樓08-09室,$101-200,越南菜,串燒,270,(380 食評),16970,訂座享晚上九時半後七九折優惠 (需消費滿$200)


In [51]:
df.to_csv('openrice_sample.csv', encoding='utf-8', index=False)